In [1]:
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import xlsxwriter as xw

%matplotlib inline

In [16]:
path = r'C:\Users\adai\Documents\PD Study Data\RawData\1004'
#path2 = r'C:\Users\adai\Documents\PD Study Data\RawData'
#path3 = r'C:\Users\adai\Documents\PD Study Data\RawData\*\*'
#print(path3)
#folderDepth = glob.glob(path3)
#dirsDepth = filter(lambda f: os.path.isdir(f), folderDepth)

#for folders in folderDepth:
    #if not folders.endswith('.csv'):
       # print(folders)

#testInfo, annotations = process_annotations(path)

df = pd.read_csv(os.path.join(path, 'annotations.csv'))
del df['Timestamp (ms)']
del df['AnnotationId']
del df['AuthorId']
    
df['Start Timestamp (ms)'] = pd.to_datetime(df['Start Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
df['Stop Timestamp (ms)'] = pd.to_datetime(df['Stop Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
df.to_csv(path + '\RawAnnotations.csv')
            
testInfo = df[df.EventType == 'Testing Day'].dropna(how='any', axis=0)
del testInfo['Stop Timestamp (ms)']
del testInfo['EventType']
del df['Value']
    
testInfo = testInfo.rename(columns = {'Value':'Day', 'Start Timestamp (ms)':'Date'}).reset_index(drop=True)
testInfo['Date'] = testInfo['Date'].dt.date
    
df = df[(df.EventType != 'Testing Day')]
    
sorter = df.EventType.unique()
sorterIndex = dict(zip(sorter,range(len(sorter))))

df['EventType_Rank'] = df['EventType'].map(sorterIndex)
df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
df[df['EventType'].str.contains('MDS-UPDRS')] = df[df['EventType'].str.contains('MDS-UPDRS')].replace(to_replace={'Cycle': {2: 3}})
df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN'}})
df.sort_values(['EventType', 'EventType_Rank', 'Start Timestamp (ms)'], axis=0)

print(testInfo)
print(df)
    
Day1 = testInfo.loc[testInfo['Day'] == 'DAY 1', 'Date']
#print(df['Start Timestamp (ms)'].dt.date == Day1)
#print(Day1)
#print(type(Day1))
#print(df['Start Timestamp (ms)'].dt.date)
#print(type(df['Start Timestamp (ms)'].dt.date))
    
    #print(df[df['Start Timestamp (ms)'].dt.date == testInfo['Date'].loc(testInfo['Day' == 'DAY 1'])])
    #name = 'NewAnnotations_' + str(testDay) + '.xlsx'
    #writer = pd.ExcelWriter(name, options={'remove_timezone': True})
    #df.to_excel(writer,'Sheet1')
    
    #return testInfo, df
        
accel = xw.Workbook(os.path.join(path, 'AccelData.xlsx'))
gyro = xw.Workbook(os.path.join(path, 'GyroData.xlsx'))
elec = xw.Workbook(os.path.join(path, 'ElecData.xlsx'))

for root, dirs, files in os.walk(path, topdown=True):
    for filenames in files:
        if filenames.endswith('accel.csv'):
            i = 1
            #filename_path = os.path.join(root, filenames)
            #print(filenames)
            #print(filename_path)
        elif filenames.endswith('elec.csv'):
            i = 1
            #filename_path = os.path.join(root, filenames)
            #print(filenames)
            #print(filename_path)
        elif filenames.endswith('gyro.csv'):
            i = 1
            #filename_path = os.path.join(root, filenames)
            #print(filenames)
            #print(filename_path)

         Date    Day
0  2017-07-07  DAY 1
1  2017-07-18  DAY 2
                                EventType             Start Timestamp (ms)  \
0                  Heart Rate Variability 2017-07-07 08:15:33.880000-05:00   
3            MDS-UPDRS #1: Finger Tapping 2017-07-07 08:32:44.865000-05:00   
4            MDS-UPDRS #2: Hand Movements 2017-07-07 08:33:01.669000-05:00   
5      MDS-UPDRS #3: Pronation-Supination 2017-07-07 08:33:15.837000-05:00   
6               MDS-UPDRS #4: Toe Tapping 2017-07-07 08:33:28.186000-05:00   
7               MDS-UPDRS #5: Leg Agility 2017-07-07 08:33:43.006000-05:00   
8        MDS-UPDRS #6: Arising from Chair 2017-07-07 08:33:53.974000-05:00   
9                      MDS-UPDRS #7: Gait 2017-07-07 08:34:05.293000-05:00   
10       MDS-UPDRS #8: Postural Stability 2017-07-07 08:34:29.293000-05:00   
11     MDS-UPDRS #9: Postural Hand Tremor 2017-07-07 08:35:05.427000-05:00   
12             MDS-UPDRS #11: Rest Tremor 2017-07-07 08:35:30.810000-05:00   
1

In [ ]:
def process_annotations(path):
    df = pd.read_csv(os.path.join(path, 'annotations.csv'))
    del df['Timestamp (ms)']
    del df['AnnotationId']
    del df['AuthorId']
    
    df['Start Timestamp (ms)'] = pd.to_datetime(df['Start Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
    df['Stop Timestamp (ms)'] = pd.to_datetime(df['Stop Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
    df.to_csv(path + '\RawAnnotations.csv')
            
    testInfo = df[df.EventType == 'Testing Day'].dropna(how='any', axis=0)
    del testInfo['Stop Timestamp (ms)']
    del testInfo['EventType']
    del df['Value']
    
    testInfo = testInfo.rename(columns = {'Value':'Day', 'Start Timestamp (ms)':'Date'}).reset_index(drop=True)
    testInfo['Date'] = testInfo['Date'].dt.date
    
    df = df[(df.EventType != 'Testing Day')]
    
    sorter = df.EventType.unique()
    sorterIndex = dict(zip(sorter,range(len(sorter))))

    df['EventType_Rank'] = df['EventType'].map(sorterIndex)
    df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
    df[df['EventType'].str.contains('MDS-UPDRS')] = df[df['EventType'].str.contains('MDS-UPDRS')].replace(to_replace={'Cycle': {2: 3}})
    df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN'}})
    df.sort_values(['EventType', 'EventType_Rank', 'Start Timestamp (ms)'], axis=0)\
    
    Day1 = testInfo.loc[testInfo['Day'] == 'DAY 1', 'Date']
    print(df['Start Timestamp (ms)'].dt.date == Day1)
    print(Day1)
    print(df['Start Timestamp (ms)'].dt.date)
    
    #print(df[df['Start Timestamp (ms)'].dt.date == testInfo['Date'].loc(testInfo['Day' == 'DAY 1'])])
    #name = 'NewAnnotations_' + str(testDay) + '.xlsx'
    #writer = pd.ExcelWriter(name, options={'remove_timezone': True})
    #df.to_excel(writer,'Sheet1')
    
    return testInfo, df

In [ ]:
path = r'C:\Users\adai\Documents\PD Study Data\RawData\1004'

for root, dirs, files in os.walk(path):
    for filenames in files:
        #if os.path.splittext(filenames)[-1] == '.csv':
        if filenames.endswith('.csv'):
            filename_path = os.path.join(root, filenames)
            print(filenames)
            print(filename_path)

#files = os.listdir(path)

#for entries in files:
#    if entries == 'annotations.csv':
#        print(os.path.join(path, entries))
#        df = pd.read_csv(os.path.join(path, entries))

        
### Annotations Processing
# removes unnecessary data columns
del df['Timestamp (ms)']
del df['AnnotationId']
del df['AuthorId']

df['Start Timestamp (ms)'] = pd.to_datetime(df['Start Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')
df['Stop Timestamp (ms)'] = pd.to_datetime(df['Stop Timestamp (ms)'], unit='ms', utc=True).dt.tz_localize('UTC').dt.tz_convert('US/Central')

testDay = df[df.EventType == 'Testing Day'].dropna(how='any').iloc[0]['Value'].replace(" ", "")

df = df[(df.EventType != 'Testing Day')]
del df['Value']

### Activity Sorting
sorter = df.EventType.unique()
sorterIndex = dict(zip(sorter,range(len(sorter))))

print(sorter)

df['EventType_Rank'] = df['EventType'].map(sorterIndex)
df['Cycle'] = df.groupby('EventType')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
df[df['EventType'].str.contains('MDS-UPDRS')] = df[df['EventType'].str.contains('MDS-UPDRS')].replace(to_replace={'Cycle': {2: 3}})
df[df['EventType'].str.contains('Heart')] = df[df['EventType'].str.contains('Heart')].replace(to_replace={'Cycle': {1: 'NaN', 2: 'NaN'}})
df.sort_values(['EventType', 'EventType_Rank', 'Start Timestamp (ms)'], axis=0)

name = 'NewAnnotations_' + str(testDay) + '.xlsx'
writer = pd.ExcelWriter(name, options={'remove_timezone': True})
df.to_excel(writer,'Sheet1')

In [ ]:
def activityCheck(activities):
# Checks the downloaded data from mc10cloud.com for missing or duplicate
# activity timestamps and returns missing and duplicate activity labels
#
# Inputs: activities - list of unique activities in the entire dataframe per day
#         testDay - desired testDay to scan
#
# Outputs: missing - list of missing activities per test day and cycle
#          duplicates - list of duplicate activity labels per test day and cycle
#          absent - list of absent (skipped or missing) activity labels per test day and cycle
#          split - list of activity labels containing two or more actual activities per day and cycle
complete = {'Heart Rate Variability', 'MDS-UPDRS #1: Finger Tapping',
           'MDS-UPDRS #2: Hand Movements', 'MDS-UPDRS #3: Pronation-Supination',
           'MDS-UPDRS #4: Toe Tapping', 'MDS-UPDRS #5: Leg Agility',
           'MDS-UPDRS #6: Arising from Chair', 'MDS-UPDRS #7: Gait',
           'MDS-UPDRS #8: Postural Stability', 'MDS-UPDRS #9: Postural Hand Tremor',
           'MDS-UPDRS #10: Kinetic Hand Tremor', 'MDS-UPDRS #11: Rest Tremor',
           'Motor #1: Standing', 'Motor #2: Walking', 'Motor #3: Walking while Counting',
           'Motor #4: Finger to Nose', 'Motor #5: Alternating Hand Movements',
           'Motor #6: Sit to Stand', 'Motor #7: Drawing on Paper',
           'Motor #8: Typing on a Computer', 'Motor #9: Nuts and Bolts',
           'Motor #10: Drinking Water', 'Motor #11: Organizing Folder',
           'Motor #12: Folding Towels', 'Motor #13: Sitting', 'Testing Day'}



In [ ]:
def merge_timestamps(df, merge):
# Inputs:  df - original annotation dataframe
#          merge - dataframe of activities to merge
#
# Outputs: df_new - updated dataframe with merged timestampes per activities
    for labels in merge:
            
        
    

In [ ]:
def absent_timestamps(df, absent):
# Inputs:  df - original annotation dataframe
#          absent - dataframe of absent (missing or skipped) activity labels
#
# Outputs: df_new - updated dataframe with placeholder activities to indicate
#                   the activity was missing
    for labels in missing:
            

In [ ]:
def split_timestamps(df, split):
# Inputs:  df - original annotation dataframe
#          split - dataframe of activity labels needed to be split
#
# Outputs: df_new - updated dataframe with split activities
    for labels in split:
        